In [25]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [26]:
mergedf = pd.read_csv('metadata_otu_merged_famID-to-str.csv')
# mergedf = pd.read_csv('metadata_otu_merged_famID-to-str_onlyFamwPet.csv')
# mergedf = pd.read_csv('metadata_otu_merged_non-rarefied.csv')
dog_fam = pd.read_csv('dogFamily_toMerge.csv')
mergedf.head(1)

C:\Users\Romina\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (2,3,24,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,...,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Nocardioidaceae; g__; s__.16,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum
0,0,797.02.HA.Forehead,1,1,years,Adolescent,FS_01,UBERON:skin,UBERON:sebum,UBERON:face,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# dog_fam.head()

In [28]:
#MERGE with DOG_FAM info
mergedf = mergedf.merge(dog_fam, left_on="family", right_on="family", suffixes=(False,False), how="outer")

mergedf["have_dog"].fillna("no",inplace = True)
# Traceback (most recent call last):
# ...
# ValueError: columns overlap but no suffix specified:
#     Index(['value'], dtype='object')
print(mergedf.shape)

(1050, 1089)


In [29]:
print(mergedf.shape)
mergedf = mergedf.drop("Unnamed: 0", axis=1)
mergedf = mergedf[mergedf.family_relationship !="none"]#EDITABLE

mergedf = mergedf[mergedf.role !="Offspring"]#EDITABLE
# mergedf = mergedf[mergedf.sample_type !="stool"]#EDITABLE

print(mergedf.shape)

(1050, 1089)
(803, 1088)


In [30]:
##FILTERING
mergedf.head(2)

,sample_name,age,age_in_years,age_units,agegroup,anonymized_name,body_habitat,body_product,body_site,collection_timestamp,...,k__Bacteria; p__Firmicutes; c__Erysipelotrichi; o__Erysipelotrichales; f__Erysipelotrichaceae; g__; s__.7,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__[Odoribacteraceae]; g__Odoribacter; s__.1,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__intermedia,k__Bacteria; p__Firmicutes; c__Bacilli; o__Lactobacillales; f__Streptococcaceae; g__Streptococcus; s__.41,k__Bacteria; p__Cyanobacteria; c__Chloroplast; o__Streptophyta; f__; g__; s__.6,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.54,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Prevotellaceae; g__Prevotella; s__.23,k__Bacteria; p__Bacteroidetes; c__Bacteroidia; o__Bacteroidales; f__Bacteroidaceae; g__Bacteroides; s__.55,k__Bacteria; p__Actinobacteria; c__Actinobacteria; o__Actinomycetales; f__Propionibacteriaceae; g__Propionibacterium; s__granulosum,have_dog
5,797.02.HB.Forehead,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:face,4/6/2010,...,2,0,0,0,0,0,0,0,0,no
6,797.02.HB.LeftPalm,35,35,years,Adult,FS_02,UBERON:skin,UBERON:sebum,UBERON:skin of hand,4/6/2010,...,0,2,0,0,1,0,0,0,0,no


In [31]:
#SPLITTING TRAIN and TEST (test = dogs, train = humans)

human_data_train = mergedf[mergedf.host_common_name =="human"]
#FILTERING OUT DOG OWNERS from Training data
human_data_train = human_data_train[human_data_train.have_dog =="yes"]


X_train = human_data_train.iloc[:, 53:1087]
y_train = human_data_train["family"].values.reshape(-1, 1)


dog_data_test = mergedf[mergedf.host_common_name =="dog"]
y_test = dog_data_test["family"].values.reshape(-1, 1)
X_test = dog_data_test.iloc[:, 53:1087]

# Hyperparameter Tuning - & Cross Validation

In [32]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [33]:
#Hyperparameter grid
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [34]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train.ravel())

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 12.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [35]:
rf_random.best_params_
# {'n_estimators': 1600,
#  'min_samples_split': 5,
#  'min_samples_leaf': 1,
#  'max_features': 'auto',
#  'max_depth': 90,
#  'bootstrap': True}

{'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': True}

In [40]:
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy


In [41]:
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train.ravel())
base_accuracy = evaluate(base_model, X_test, y_test.ravel())

Model Performance
Average Error: 16.5127 degrees.
Accuracy = -22.14%.


In [43]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test.ravel())

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
print(random_accuracy)

Model Performance
Average Error: 16.8039 degrees.
Accuracy = -24.51%.
Improvement of 10.69%.
-24.50915099464237


# GRID search with Cross Validation

In [44]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [45]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train.ravel())
grid_search.best_params_

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test.ravel())

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))


Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  2.1min finished


Model Performance
Average Error: 18.9487 degrees.
Accuracy = -33.16%.
Improvement of 49.77%.


In [46]:
best_grid

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=80, max_features=3, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=3,
                      min_samples_split=8, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [8]:
# print(mergedf.iloc[:,[1087]])

# y = mergedf["familyID"].values.reshape(-1, 1)
# print(X.shape, y.shape)
# X_test.head(1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
#from non-scaled data
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf = rf.fit(X_train, y_train.ravel())
rf.score(X_test, y_test) #geni impurity coefficient

0.19248826291079812

In [11]:
# from scaled data
rf_scaled = RandomForestClassifier(n_estimators=500)
rf_scaled = rf_scaled.fit(X_train_scaled, y_train.ravel())
print(f'Test score:{rf_scaled.score(X_test_scaled, y_test)}') #geni impurity coefficient
print(f'Train score: {rf_scaled.score(X_train_scaled, y_train.ravel())}')

Test score:0.18309859154929578
Train score: 1.0


# PLAYING WITH FILTERING

In [12]:
mergedf_filter = mergedf#[mergedf.role =="Partner"]
mergedf_filter = mergedf_filter[mergedf_filter.sample_type !="stool"]

X_f = mergedf_filter.iloc[:, 53:1085]
y_f = mergedf_filter["familyID"].values.reshape(-1, 1)
print(X_f.shape, y_f.shape)

(648, 1032) (648, 1)


In [13]:
human_data_train = mergedf_filter[mergedf_filter.host_common_name =="human"]
dog_data_test = mergedf_filter[mergedf_filter.host_common_name =="dog"]

X_train = human_data_train.iloc[:, 53:1085]
X_test = dog_data_test.iloc[:, 53:1085]
y_train = human_data_train["familyID"].values.reshape(-1, 1)
y_test = dog_data_test["familyID"].values.reshape(-1, 1)

In [14]:
#only looking at human families, predict which family each person belongs to
human_data = mergedf_filter[mergedf_filter.family_relationship !="none"]
human_data = mergedf_filter[mergedf_filter.host_common_name =="human"]
human_data = mergedf_filter[mergedf_filter.role !="offspring"]

X = mergedf.iloc[:, 53:1085]
y = mergedf["familyID"].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train.ravel())
print("test")
print(rf.score(X_test, y_test)) #geni impurity coefficient
print("train")
print(rf.score(X_train, y_train))

test
0.3383084577114428
train
1.0
